<a href="https://colab.research.google.com/github/RohitGanji/NN-Digit_Recognizer/blob/main/Digit%20Recognizer%20-%20CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [ ]:
# Load data

import pandas as pd

train = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
test = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")

In [ ]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Reshape and normalize
X = train.drop('label', axis = 1)
y = train['label']

# Normalize and Reshape
X = X.astype('float32') / 255.0
X = X.values.reshape(-1, 28, 28, 1)

X_test = test.astype('float32') / 255.0
X_test = X_test.values.reshape(-1, 28, 28, 1)

In [ ]:
# Split into train and test data

from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(33600, 28, 28, 1) (8400, 28, 28, 1) (33600,) (8400,)


In [ ]:
# Convert y to categorical

import tensorflow as tf

classes = len(np.unique(y_train))
print(classes)

y_train = tf.keras.utils.to_categorical(y_train, classes)
y_val = tf.keras.utils.to_categorical(y_val, classes)

10


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

callbacks = [
    EarlyStopping(monitor="val_accuracy",
                 min_delta=1e-4,
                 patience=5),
    ModelCheckpoint(filepath="model.h5",
                   monitor="val_accuracy",
                   mode="max",
                   save_best_only=True)
]

In [ ]:
# Model

model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same', input_shape=(28,28,1)),
  tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same'),
  tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPooling2D(2),
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same'),
  tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same'),
  tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPooling2D(2),
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(loss = tf.keras.losses.categorical_crossentropy,
              optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4),
              metrics = ['accuracy'])

history = model.fit(X_train, y_train,
                    validation_data = (X_val, y_val),
                    epochs=500, batch_size=64,
                    callbacks=[callbacks])

Epoch 1/500
525/525 [==============================] - 7s 12ms/step - loss: 0.9008 - accuracy: 0.7236 - val_loss: 1.7278 - val_accuracy: 0.5476
Epoch 2/500
525/525 [==============================] - 6s 11ms/step - loss: 0.1283 - accuracy: 0.9617 - val_loss: 0.0684 - val_accuracy: 0.9790
Epoch 3/500
525/525 [==============================] - 6s 11ms/step - loss: 0.0757 - accuracy: 0.9757 - val_loss: 0.0517 - val_accuracy: 0.9839
Epoch 4/500
525/525 [==============================] - 6s 11ms/step - loss: 0.0577 - accuracy: 0.9819 - val_loss: 0.0427 - val_accuracy: 0.9857
Epoch 5/500
525/525 [==============================] - 6s 12ms/step - loss: 0.0515 - accuracy: 0.9828 - val_loss: 0.0462 - val_accuracy: 0.9857
Epoch 6/500
525/525 [==============================] - 6s 11ms/step - loss: 0.0418 - accuracy: 0.9871 - val_loss: 0.0404 - val_accuracy: 0.9882
Epoch 7/500
525/525 [==============================] - 6s 11ms/step - loss: 0.0373 - accuracy: 0.9890 - val_loss: 0.0336 - val_accuracy:

In [ ]:
model = tf.keras.models.load_model("model.h5")

In [ ]:
model.evaluate(X_val, y_val)

263/263 [==============================] - 1s 4ms/step - loss: 0.0282 - accuracy: 0.9926


[0.028215475380420685, 0.9926190376281738]

In [ ]:
preds = model.predict(X_test)
preds[:5]

array([[4.66076031e-08, 5.32387956e-09, 1.00000000e+00, 2.51881325e-08,
        1.59261528e-08, 1.34408651e-09, 1.98349692e-09, 1.09369429e-08,
        8.28633695e-09, 1.71181946e-09],
       [9.99998689e-01, 2.83576291e-08, 7.88675266e-07, 7.04655800e-09,
        1.52917179e-09, 6.46969340e-08, 2.98268503e-07, 3.02101562e-08,
        9.99321514e-09, 5.55009017e-08],
       [6.39567645e-07, 2.41763416e-07, 2.88542560e-05, 8.93997367e-07,
        3.96508316e-04, 1.63320479e-07, 9.17079390e-08, 1.29670767e-07,
        1.41037075e-04, 9.99431431e-01],
       [8.73131037e-01, 9.34101934e-07, 3.29408725e-03, 1.72665215e-03,
        5.62272817e-06, 7.99833651e-05, 7.79396680e-04, 1.78685586e-04,
        7.77681940e-04, 1.20025985e-01],
       [5.11295397e-08, 3.20792779e-08, 1.38093610e-05, 9.99977827e-01,
        1.61300733e-08, 8.35159213e-08, 6.80244838e-08, 1.04533491e-07,
        7.66951962e-06, 2.91471167e-07]], dtype=float32)

In [ ]:
results = tf.argmax(preds, axis=1)
results[:5]

<tf.Tensor: shape=(5,), dtype=int64, numpy=array([2, 0, 9, 0, 3])>

In [ ]:
submit = pd.read_csv("/kaggle/input/digit-recognizer/sample_submission.csv")
submit['Label'] = results
submit.to_csv("submit.csv", index=False)